In [1]:
#!/usr/bin/env python3
"""
exit_checker.py

Reads buy_signals.csv produced by the scanner, checks exit rules for each ticker,
and sends Telegram alerts with reasons when an exit is recommended.

- Auto-detects latest outputs/YYYY-MM-DD/buy_signals.csv if INPUT_PATH=None.
- Exit rules (configurable):
    * STOP_LOSS_PCT: exit if Close <= entry_price * (1 - stop_loss_pct)
    * TRAIL_ATR:     exit if Close <= (highest_close_since_entry - ATR*mult)
    * MAX_HOLD_BARS: exit if bars_held >= max_hold_bars (trading bars)
    * EMA_BEAR:      exit if EMA_fast < EMA_slow
    * SMA30_DROP:    exit if Close < SMA(30)
    * SUPERTREND_DN: exit if supertrend_direction turns -1

- Telegram: uses env TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID if not set in config.
"""

import os
import re
import glob
import datetime as dt
from typing import Dict, Any, List, Optional, Tuple

import pandas as pd
import numpy as np
import yfinance as yf

# Optional third-party: requests for Telegram
try:
    import requests
except Exception:
    requests = None

try:
    from zoneinfo import ZoneInfo
except Exception:
    ZoneInfo = None


# =========================
# Config
# =========================
CONFIG: Dict[str, Any] = {
    # Path to buy_signals.csv (leave None to auto-pick latest under outputs/YYYY-MM-DD/)
    "INPUT_PATH": None,  # e.g., "outputs/2025-09-20/buy_signals.csv"

    # Data & indicators
    "YF_LOOKBACK_PAD_DAYS": 30,   # extra history before entry date for indicators
    "EMA_FAST_LENGTH": 9,
    "EMA_SLOW_LENGTH": 15,
    "SMA30_LENGTH": 30,
    "ATR_LENGTH": 14,
    "ST_LENGTH": 10,
    "ST_MULTIPLIER": 3.0,

    # Exit rule toggles
    "EXITS": {
        "STOP_LOSS_PCT": True,
        "TRAIL_ATR": True,
        "MAX_HOLD_BARS": True,
        "EMA_BEAR": True,
        "SMA30_DROP": True,
        "SUPERTREND_DN": True,
    },

    # Exit rule params
    "STOP_LOSS_PCT": 0.05,   # 5% below entry
    "TRAIL_ATR_MULT": 2.5,   # Close <= HighestCloseSinceEntry - ATR*mult
    "MAX_HOLD_BARS": 10,     # trading bars

    # Exit decision combining: 'any' (recommended) or 'all'
    "COMBINE_MODE": "any",

    # Telegram settings
    "TELEGRAM": {
        "ENABLE": True,
        "BOT_TOKEN": "",      # or env TELEGRAM_BOT_TOKEN
        "CHAT_ID": "",        # or env TELEGRAM_CHAT_ID
        "PARSE_MODE": "HTML",
        "DISABLE_WEB_PREVIEW": True,
        "DRY_RUN": False,     # force print instead of sending
        "SEND_SUMMARY": True, # one summary after per-ticker alerts
    },

    # Output folder for today’s run
    "OUTPUT_ROOT": "outputs",
}

VERBOSE = True


# =========================
# Utils
# =========================
def _now_ist_str():
    try:
        if ZoneInfo:
            return dt.datetime.now(ZoneInfo("Asia/Kolkata")).strftime("%Y-%m-%d %H:%M")
    except Exception:
        pass
    return dt.datetime.now().strftime("%Y-%m-%d %H:%M")

def _escape_html(s: str) -> str:
    return (str(s).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;"))

def resolve_telegram_creds(tcfg: Dict[str, Any]) -> Tuple[str, str]:
    token = tcfg.get("BOT_TOKEN") or os.getenv("TELEGRAM_BOT_TOKEN", "")
    chat_id = tcfg.get("CHAT_ID") or os.getenv("TELEGRAM_CHAT_ID", "")
    return token, chat_id

def send_telegram(text: str, tcfg: Dict[str, Any]):
    token, chat_id = resolve_telegram_creds(tcfg)
    if not tcfg.get("ENABLE", True):
        if VERBOSE: print("[Telegram] Disabled. Message:\n", text)
        return
    if tcfg.get("DRY_RUN", False) or (not token) or (not chat_id) or (requests is None):
        print("[Telegram] (dry-run/creds-missing/requests-missing) Would send:\n", text)
        return

    url = f"https://api.telegram.org/bot{token}/sendMessage"
    try:
        resp = requests.post(
            url,
            data={
                "chat_id": chat_id,
                "text": text,
                "parse_mode": tcfg.get("PARSE_MODE", "HTML"),
                "disable_web_page_preview": "true" if tcfg.get("DISABLE_WEB_PREVIEW", True) else "false",
            },
            timeout=15,
        )
        if resp.status_code != 200:
            print(f"[Telegram] Error {resp.status_code}: {resp.text[:200]}")
    except Exception as e:
        print(f"[Telegram] Exception sending message: {e}")

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)


# =========================
# Indicators
# =========================
def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def sma(series: pd.Series, length: int) -> pd.Series:
    return series.rolling(window=length, min_periods=1).mean()

def compute_atr(df: pd.DataFrame, length=14) -> pd.Series:
    tr = pd.concat([
        (df["High"] - df["Low"]).abs(),
        (df["High"] - df["Close"].shift()).abs(),
        (df["Low"] - df["Close"].shift()).abs(),
    ], axis=1).max(axis=1)
    return tr.rolling(length, min_periods=1).mean()

def supertrend_fallback(df: pd.DataFrame, period=10, multiplier=3.0) -> pd.DataFrame:
    hl2 = (df['High'] + df['Low']) / 2
    atr = compute_atr(df, period)
    upperband = hl2 + multiplier * atr
    lowerband = hl2 - multiplier * atr

    final_upper = upperband.copy().reset_index(drop=True)
    final_lower = lowerband.copy().reset_index(drop=True)
    close_vals = df['Close'].reset_index(drop=True)

    for i in range(1, len(df)):
        final_upper.iat[i] = upperband.iat[i] if (upperband.iat[i] < final_upper.iat[i-1]) or (close_vals.iat[i-1] > final_upper.iat[i-1]) else final_upper.iat[i-1]
        final_lower.iat[i] = lowerband.iat[i] if (lowerband.iat[i] > final_lower.iat[i-1]) or (close_vals.iat[i-1] < final_lower.iat[i-1]) else final_lower.iat[i-1]

    supertrend = pd.Series(index=df.index, dtype=float)
    direction = pd.Series(index=df.index, dtype=int)
    for i in range(len(df)):
        if i == 0 or df['Close'].iat[i] <= final_upper.iat[i]:
            supertrend.iat[i] = final_upper.iat[i]
            direction.iat[i] = -1
        else:
            supertrend.iat[i] = final_lower.iat[i]
            direction.iat[i] = 1

    out = df.copy()
    out['supertrend'] = supertrend
    out['supertrend_direction'] = direction
    return out


# =========================
# Core
# =========================
def locate_latest_buy_signals() -> Optional[str]:
    """Pick newest outputs/YYYY-MM-DD/buy_signals.csv by date in folder name."""
    candidates = sorted(glob.glob(os.path.join("outputs", "20??-??-??", "buy_signals.csv")))
    return candidates[-1] if candidates else None

def read_buy_signals(input_path: Optional[str]) -> pd.DataFrame:
    path = input_path or locate_latest_buy_signals()
    if not path or not os.path.exists(path):
        raise FileNotFoundError("buy_signals.csv not found. Set CONFIG['INPUT_PATH'] or ensure outputs/YYYY-MM-DD/buy_signals.csv exists.")
    df = pd.read_csv(path)
    # Normalize expected columns
    # Prefer 'entry_price' & 'entry_date' if present, else fallback to last_close/last_bar_date
    if 'entry_price' not in df.columns and 'last_close' in df.columns:
        df['entry_price'] = df['last_close']
    if 'entry_date' not in df.columns and 'last_bar_date' in df.columns:
        df['entry_date'] = df['last_bar_date']
    if 'ticker' not in df.columns:
        # sometimes 'symbol' is used
        if 'symbol' in df.columns:
            df = df.rename(columns={'symbol': 'ticker'})
        else:
            raise ValueError("Input CSV must contain 'ticker' or 'symbol' column.")
    return df[['ticker', 'entry_price', 'entry_date']].dropna()

def fetch_history(ticker: str, start_date: dt.date, pad_days: int) -> pd.DataFrame:
    start = (start_date - dt.timedelta(days=pad_days)).strftime("%Y-%m-%d")
    end = (dt.date.today() + dt.timedelta(days=1)).strftime("%Y-%m-%d")
    df = yf.download(ticker, start=start, end=end, interval="1d", progress=False, auto_adjust=True, multi_level_index=False)
    if df is None or df.empty:
        return pd.DataFrame()
    df.index = pd.to_datetime(df.index).tz_localize(None)
    return df

def evaluate_exits_for_ticker(
    ticker: str,
    entry_price: float,
    entry_date_str: str,
    cfg: Dict[str, Any]
) -> Dict[str, Any]:
    """Return dict with exit boolean and reasons."""
    try:
        entry_date = pd.to_datetime(entry_date_str).date()
    except Exception:
        # If parsing fails, assume we entered on the most recent bar in past
        entry_date = (dt.date.today() - dt.timedelta(days=1))

    df = fetch_history(ticker, entry_date, cfg["YF_LOOKBACK_PAD_DAYS"])
    if df.empty or len(df) < 2:
        return {"ticker": ticker, "exit": False, "reason": "insufficient_data"}

    close = df['Close']
    high = df['High']
    low = df['Low']

    # Indicators
    df['EMA_fast'] = ema(close, cfg["EMA_FAST_LENGTH"])
    df['EMA_slow'] = ema(close, cfg["EMA_SLOW_LENGTH"])
    df['SMA_30']   = sma(close, cfg["SMA30_LENGTH"])
    df['ATR']      = compute_atr(df, cfg["ATR_LENGTH"])
    st_df          = supertrend_fallback(df, cfg["ST_LENGTH"], cfg["ST_MULTIPLIER"])
    df[['supertrend', 'supertrend_direction']] = st_df[['supertrend', 'supertrend_direction']]

    # Subset from entry date onward (inclusive)
    df_after = df.loc[df.index.date >= entry_date].copy()
    if df_after.empty:
        return {"ticker": ticker, "exit": False, "reason": "no_bars_after_entry"}

    # Bars held & current state
    bars_held = len(df_after)
    cur = df_after.iloc[-1]
    cur_close = float(cur['Close'])

    # Highest close since entry for trailing
    highest_close = float(df_after['Close'].cummax().iloc[-1])
    last_atr = float(df_after['ATR'].iloc[-1])

    # Rules
    reasons = []

    if CONFIG["EXITS"].get("STOP_LOSS_PCT", True):
        sl_level = entry_price * (1.0 - CONFIG["STOP_LOSS_PCT"])
        if cur_close <= sl_level:
            reasons.append(f"STOP_LOSS_PCT: Close {cur_close:.2f} ≤ {sl_level:.2f} (−{CONFIG['STOP_LOSS_PCT']*100:.1f}%)")

    if CONFIG["EXITS"].get("TRAIL_ATR", True):
        trail_level = highest_close - CONFIG["TRAIL_ATR_MULT"] * last_atr
        if cur_close <= trail_level:
            reasons.append(f"TRAIL_ATR: Close {cur_close:.2f} ≤ HighSinceEntry {highest_close:.2f} − {CONFIG['TRAIL_ATR_MULT']}×ATR {last_atr:.2f} = {trail_level:.2f}")

    if CONFIG["EXITS"].get("MAX_HOLD_BARS", True):
        if bars_held >= CONFIG["MAX_HOLD_BARS"]:
            reasons.append(f"MAX_HOLD_BARS: Held {bars_held} bars ≥ {CONFIG['MAX_HOLD_BARS']}")

    if CONFIG["EXITS"].get("EMA_BEAR", True):
        if cur['EMA_fast'] < cur['EMA_slow']:
            reasons.append(f"EMA_BEAR: EMA{CONFIG['EMA_FAST_LENGTH']:.0f} < EMA{CONFIG['EMA_SLOW_LENGTH']:.0f}")

    if CONFIG["EXITS"].get("SMA30_DROP", True):
        if cur_close < cur['SMA_30']:
            reasons.append(f"SMA30_DROP: Close {cur_close:.2f} < SMA{CONFIG['SMA30_LENGTH']} {float(cur['SMA_30']):.2f}")

    if CONFIG["EXITS"].get("SUPERTREND_DN", True):
        if int(cur.get('supertrend_direction', 1)) == -1:
            reasons.append("SUPERTREND_DN: supertrend_direction turned down")

    # Combine
    exit_flag = False
    if reasons:
        mode = CONFIG.get("COMBINE_MODE", "any").lower()
        if mode == "any":
            exit_flag = True
        elif mode == "all":
            # require all enabled rules to be triggered
            enabled = [k for k, v in CONFIG["EXITS"].items() if v]
            # crude: if all enabled appear in reasons text; instead check per rule explicitly
            # We'll compute count of triggered among enabled:
            triggered = 0
            if CONFIG["EXITS"].get("STOP_LOSS_PCT", True) and "STOP_LOSS_PCT" in " ".join(reasons): triggered += 1
            if CONFIG["EXITS"].get("TRAIL_ATR", True)     and "TRAIL_ATR" in " ".join(reasons): triggered += 1
            if CONFIG["EXITS"].get("MAX_HOLD_BARS", True) and "MAX_HOLD_BARS" in " ".join(reasons): triggered += 1
            if CONFIG["EXITS"].get("EMA_BEAR", True)      and "EMA_BEAR" in " ".join(reasons): triggered += 1
            if CONFIG["EXITS"].get("SMA30_DROP", True)    and "SMA30_DROP" in " ".join(reasons): triggered += 1
            if CONFIG["EXITS"].get("SUPERTREND_DN", True) and "SUPERTREND_DN" in " ".join(reasons): triggered += 1
            exit_flag = (triggered == len(enabled))

    pnl_pct = (cur_close / entry_price - 1.0) * 100.0

    return {
        "ticker": ticker,
        "exit": bool(exit_flag),
        "reasons": "; ".join(reasons),
        "as_of": df_after.index[-1],
        "entry_price": float(entry_price),
        "current_close": cur_close,
        "pnl_pct": float(pnl_pct),
        "bars_held": int(bars_held),
        "highest_close_since_entry": float(highest_close),
        "atr_last": last_atr,
    }


# =========================
# Run
# =========================
def main():
    # Read inputs
    signals_df = read_buy_signals(CONFIG["INPUT_PATH"])

    # Output dir for this run
    run_date_str = dt.date.today().strftime("%Y-%m-%d")
    out_dir = os.path.join(CONFIG["OUTPUT_ROOT"], run_date_str)
    ensure_dir(out_dir)

    results: List[Dict[str, Any]] = []
    exits_triggered: List[Dict[str, Any]] = []

    for _, row in signals_df.iterrows():
        ticker = row["ticker"]
        entry_price = float(row["entry_price"])
        entry_date = str(row["entry_date"])

        if VERBOSE:
            print(f"Evaluating exit for {ticker} ...", end="", flush=True)

        try:
            res = evaluate_exits_for_ticker(ticker, entry_price, entry_date, CONFIG)
            results.append(res)
            if res["exit"]:
                exits_triggered.append(res)
                if VERBOSE: print(" EXIT ✅")
            else:
                if VERBOSE: print(" hold")
        except Exception as e:
            if VERBOSE: print(f" error: {e}")
            results.append({"ticker": ticker, "exit": False, "reasons": f"error: {e}"})

    # Save CSVs
    all_path = os.path.join(out_dir, "exit_evaluations.csv")
    exits_path = os.path.join(out_dir, "exit_alerts.csv")

    pd.DataFrame(results).to_csv(all_path, index=False)
    pd.DataFrame(exits_triggered).to_csv(exits_path, index=False)

    print(f"\nSaved: {all_path}")
    print(f"Exits: {len(exits_triggered)} / {len(results)}")

    # Telegram alerts
    tcfg = CONFIG["TELEGRAM"]
    if tcfg.get("ENABLE", True):
        # Per-exit alert
        for res in exits_triggered:
            text = (
                f"<b>EXIT ALERT</b> — {_escape_html(_now_ist_str())}\n"
                f"Ticker: <b>{_escape_html(res['ticker'])}</b>\n"
                f"Entry:  <b>{res['entry_price']:.2f}</b>\n"
                f"Now:    <b>{res['current_close']:.2f}</b>\n"
                f"PnL:    <b>{res['pnl_pct']:.2f}%</b>\n"
                f"Held:   <b>{res['bars_held']}</b> bars\n"
                f"Reason: {_escape_html(res['reasons'])}"
            )
            send_telegram(text, tcfg)

        # Optional summary
        if tcfg.get("SEND_SUMMARY", True):
            if exits_triggered:
                lines = []
                for res in exits_triggered:
                    lines.append(
                        f"• <b>{_escape_html(res['ticker'])}</b> "
                        f"@ {_escape_html(res['as_of'])} | "
                        f"{res['current_close']:.2f} | "
                        f"{res['pnl_pct']:.2f}% | "
                        f"{_escape_html(res['reasons'])}"
                    )
                summary = (
                    f"<b>Exit Summary</b> — {_escape_html(_now_ist_str())}\n"
                    f"Total exits: <b>{len(exits_triggered)}</b>\n\n" + "\n".join(lines)
                )
            else:
                summary = f"<b>Exit Summary</b> — {_escape_html(_now_ist_str())}\nNo exits today."
            send_telegram(summary, tcfg)


if __name__ == "__main__":
    main()


Evaluating exit for SBILIFE.NS ... EXIT ✅

Saved: outputs/2025-09-21/exit_evaluations.csv
Exits: 1 / 1
[Telegram] (dry-run/creds-missing/requests-missing) Would send:
 <b>EXIT ALERT</b> — 2025-09-21 15:56
Ticker: <b>SBILIFE.NS</b>
Entry:  <b>1841.70</b>
Now:    <b>1841.70</b>
PnL:    <b>0.00%</b>
Held:   <b>1</b> bars
Reason: SUPERTREND_DN: supertrend_direction turned down
[Telegram] (dry-run/creds-missing/requests-missing) Would send:
 <b>Exit Summary</b> — 2025-09-21 15:56
Total exits: <b>1</b>

• <b>SBILIFE.NS</b> @ 2025-09-19 00:00:00 | 1841.70 | 0.00% | SUPERTREND_DN: supertrend_direction turned down
